<a href="https://colab.research.google.com/github/Kossisamuel/planets/blob/main/AFOLA_Kossi_Bot_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Author: AFOLA Kossi Mawouena Samuel, 2025
## My Chat Bot1.0,
## RAG

In [1]:
import warnings
warnings.filterwarnings('ignore')

From Kaggle!

In [2]:
!kaggle datasets download adilmohammed/medical-data

Dataset URL: https://www.kaggle.com/datasets/adilmohammed/medical-data
License(s): MIT
  0% 0.00/13.8k [00:00<?, ?B/s]
100% 13.8k/13.8k [00:00<00:00, 24.9MB/s]


In [3]:
!unzip medical-data

Archive:  medical-data.zip
  inflating: medical_data.csv        


In [4]:
import pandas as pd
df = pd.read_csv('./medical_data.csv')
df.head()

,Patient_Problem,Disease,Prescription
0,"Constant fatigue and muscle weakness, struggli...",Chronic Fatigue Syndrome,"Cognitive behavioral therapy, graded exercise ..."
1,"Frequent severe migraines, sensitivity to ligh...",Migraine with Aura,"Prescription triptans, avoid triggers like bri..."
2,"Sudden weight gain and feeling cold, especiall...",Hypothyroidism,Levothyroxine to regulate thyroid hormone levels.
3,"High fever, sore throat, and swollen lymph nod...",Mononucleosis,"Rest and hydration, ibuprofen for pain."
4,"Excessive thirst and frequent urination, dry m...",Diabetes Mellitus,Insulin therapy and lifestyle changes.


# Form context from tables

In [5]:
df.columns

Index(['Patient_Problem', 'Disease', 'Prescription'], dtype='object')

In [6]:

context_data = []
for i in range(len(df)):
  context = ""
  for j in range(3):
    context += df.columns[j]
    context += ": "
    context += df.iloc[i][j]
    context += " "
  context_data.append(context)

In [7]:
len(context_data)

407

In [8]:
context_data[0]

'Patient_Problem: Constant fatigue and muscle weakness, struggling to stay awake. Disease: Chronic Fatigue Syndrome Prescription: Cognitive behavioral therapy, graded exercise therapy. '

In [9]:
# Configuration pour saisir la clé API
from getpass import getpass
groq_api_key = getpass('Entrez votre clé API : ')

# Affichez les cinq premiers et derniers caractères pour la vérification
print("Clé API configurée :", groq_api_key[:5] + "..." + groq_api_key[-5:])


Entrez votre clé API : ··········
Clé API configurée : gsk_B...VxbeA


In [10]:
%%capture
!pip install langchain_groq langchain_huggingface langchain_chroma

In [11]:
## LLM used for RAG
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.3-70b-versatile",api_key=groq_api_key)

In [12]:
## Embedding model!
from langchain_huggingface import HuggingFaceEmbeddings
embed_model = HuggingFaceEmbeddings(model_name="mixedbread-ai/mxbai-embed-large-v1")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/114k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [13]:
# create vector store!
from langchain_chroma import Chroma

vectorstore = Chroma(
    collection_name="medical_dataset_store",
    embedding_function=embed_model,
    persist_directory="./",
)


In [14]:
vectorstore.get().keys()

dict_keys(['ids', 'embeddings', 'documents', 'uris', 'data', 'metadatas', 'included'])

In [15]:
# add data to vector nstore
%%capture
vectorstore.add_texts(context_data)

In [16]:
query = "What is the disease for constant fatigue and muscle weakness?"
docs = vectorstore.similarity_search(query)
print(docs[0].page_content)

Patient_Problem: Constant fatigue and muscle weakness, struggling to stay awake. Disease: Chronic Fatigue Syndrome Prescription: Cognitive behavioral therapy, graded exercise therapy. 


In [17]:
retriever = vectorstore.as_retriever()

In [18]:
from langchain_core.prompts import PromptTemplate

template = ("""You are a medical expert.
    Use the provided context to answer the question.
    If you don't know the answer, say so. Explain your answer in detail.
    Do not discuss the context in your response; just provide the answer directly.

    Context: {context}

    Question: {question}

    Answer:""")

rag_prompt = PromptTemplate.from_template(template)

In [19]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

# Example of the questions



1.   What is the disease for constant fatigue and muscle weakness?
2.   What is Capital of Togo?
3.   Eplain me muscle weakness in french



In [20]:
from IPython.display import display, Markdown


user_question = input("Enter your question : ")


response = rag_chain.invoke(user_question)

display(Markdown(response))


Enter your question : What is the disease for constant fatigue and muscle weakness?


The disease for constant fatigue and muscle weakness is Chronic Fatigue Syndrome. This condition is characterized by persistent and unexplained fatigue that is not relieved by rest, as well as muscle weakness, which can significantly impact a person's quality of life. The symptoms of constant fatigue and muscle weakness are consistent with the diagnosis of Chronic Fatigue Syndrome, which is a chronic and complex condition that requires a comprehensive treatment approach, including cognitive behavioral therapy, graded exercise therapy, and other management strategies to help alleviate symptoms and improve functioning.

In [21]:
user_question = input("Enter your question: ")

# Utiliser la question saisie dans le modèle RAG
response = rag_chain.invoke(user_question)

# Afficher la réponse en Markdown
display(Markdown(response))

Enter your question: Eplain me muscle weakness in french


La faiblesse musculaire se réfère à une diminution de la force ou de la puissance des muscles, ce qui peut entraîner des difficultés à effectuer des tâches quotidiennes. Elle peut être causée par divers facteurs, tels que des maladies neurologiques, des troubles musculaires, des carences nutritionnelles ou des médicaments. Les symptômes peuvent inclure une diminution de la force musculaire, des crampes, des spasmes ou des douleurs musculaires. Le traitement de la faiblesse musculaire dépend de la cause sous-jacente et peut inclure des exercices de renforcement musculaire, des médicaments pour soulager les symptômes, ou des thérapies pour améliorer la fonction musculaire. 

Dans le contexte des maladies mentionnées, la faiblesse musculaire peut être un symptôme de la dystrophie musculaire, une maladie génétique qui affecte les muscles et entraîne une perte progressive de la force musculaire. Elle peut également être un symptôme de la fatigue chronique, une maladie caractérisée par une fatigue persistante et une faiblesse musculaire généralisée. 

En français, on utilise également des termes plus spécifiques pour décrire la faiblesse musculaire, tels que "amyotrophie" pour décrire la perte de masse musculaire, ou "myasthénie" pour décrire la faiblesse musculaire due à un dysfonctionnement de la transmission nerveuse. 

En résumé, la faiblesse musculaire est un symptôme complexe qui peut avoir de multiples causes et manifestations, et son traitement doit être adapté à la cause sous-jacente et aux besoins individuels du patient.

In [22]:
## Deployment

In [23]:
%%capture
!pip install gradio

In [24]:
import gradio as gr

def rag_memory_stream(text):
    partial_text = ""
    for new_text in rag_chain.stream(text):
        partial_text += new_text
        yield partial_text


title = "Real-time AI App with Groq API and LangChain to Answer medical questions"
demo = gr.Interface(
    title=title,
    fn=rag_memory_stream,
    inputs="text",
    outputs="text",
    allow_flagging="never",
)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://204bc57490f5d28700.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
